In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Data exploration and analysis preparation

#### Reading the data and converting them into a Dataframe 

In [ ]:
# Load all dataframes 

from data_loader import load_dataset as load_all

other_road_users, sensors = load_all()
print('Loaded', len(other_road_users), 'other dataframes and', len(sensors), 'sensors dataframes')
print('Sample other_road_users keys:', list(other_road_users.keys())[:10])
print('Sample sensors keys:', list(sensors.keys())[:10])

Loaded 12 other dataframes and 12 sensors dataframes
Sample other_road_users keys: ['ebike_subject_a', 'ebike_subject_b', 'escooter_subject_b', 'escooter_subject_c', 'ebike_subject_e', 'escooter_subject_e', 'ebike_subject_g', 'escooter_subject_g', 'ebike_subject_h', 'escooter_subject_h']
Sample sensors keys: ['ebike_subject_a', 'ebike_subject_b', 'escooter_subject_b', 'escooter_subject_c', 'ebike_subject_e', 'escooter_subject_e', 'ebike_subject_g', 'escooter_subject_g', 'ebike_subject_h', 'escooter_subject_h']


In [6]:
other_road_users['ebike_subject_a']

,frame_index,track_id,class_name,angle,distance,x,y,x_inter,y_inter,x_inter_rts,y_inter_rts,vx_rts,vy_rts,interpolated,corrected_class
0,322,4443,Pedestrian,-14.614138,14.985338,-3.8,14.5,-3.8,14.5,-4.1,14.1,-1.3,-6.6,False,Pedestrian
1,323,4443,Pedestrian,-15.551180,14.918787,-4.0,14.4,-4.0,14.4,-4.1,13.8,-1.3,-6.7,False,Pedestrian
2,324,4443,Pedestrian,-16.408048,14.435930,-4.1,13.8,-4.1,13.8,-4.2,13.6,-1.3,-6.7,False,Pedestrian
3,325,4443,Pedestrian,-17.254617,14.299149,-4.2,13.7,-4.2,13.7,-4.3,13.3,-1.2,-6.6,False,Pedestrian
4,326,4443,Pedestrian,-18.103867,13.987128,-4.3,13.3,-4.3,13.3,-4.3,13.0,-1.2,-6.6,False,Pedestrian
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1502,953,4736,Pedestrian,47.627269,3.997782,3.0,2.7,3.0,2.7,3.0,2.5,-1.6,-9.3,False,Pedestrian
1503,954,4736,Pedestrian,51.310109,3.514907,2.7,2.2,2.7,2.2,2.9,2.1,-1.6,-9.3,False,Pedestrian
1504,955,4736,Pedestrian,54.713091,3.473502,2.8,2.0,2.8,2.0,2.8,1.8,-1.6,-9.3,False,Pedestrian
1505,956,4736,Pedestrian,58.906697,3.266233,2.8,1.7,2.8,1.7,2.8,1.4,-1.6,-9.2,False,Pedestrian
